In [ ]:
# 📦 STANDARD LIBRARY IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import yaml
from pathlib import Path
from datetime import datetime

# 🎨 VISUALIZATION SETUP
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("✅ Standard libraries loaded successfully!")
print("🎯 Ready for universal experiment analysis")


In [ ]:
# 🔍 EXPERIMENT DATA LOADING
# This section automatically finds and loads experiment data

# Find experiment data files in current directory
current_dir = Path('.')
run_metadata_file = current_dir / 'run_metadata.json'

# Load run metadata if available
if run_metadata_file.exists():
    with open(run_metadata_file, 'r') as f:
        run_metadata = json.load(f)
    print(f"✅ Run metadata loaded: {run_metadata.get('job_id', 'Unknown')}")
    
    # Find experiment YAML file
    experiment_path = run_metadata.get('experiment_path')
    if experiment_path and Path(experiment_path).exists():
        with open(experiment_path, 'r') as f:
            experiment_def = yaml.safe_load(f)
        print(f"✅ Experiment definition loaded: {experiment_def.get('name', 'Unknown')}")
    else:
        print("⚠️ Experiment YAML not found - using metadata only")
        experiment_def = {}
else:
    print("⚠️ No run metadata found")
    print("💡 Tip: This template works best with experiment results")
    run_metadata = {}
    experiment_def = {}

# Display basic experiment info
print("\n📊 Experiment Overview:")
print(f"   Name: {experiment_def.get('name', 'Unknown')}")
print(f"   Framework: {experiment_def.get('framework', {}).get('name', 'Unknown')}")
print(f"   Models: {run_metadata.get('models_used', ['Not specified'])}")
print(f"   Job ID: {run_metadata.get('job_id', 'Unknown')}")


In [ ]:
# 📈 RESULTS DATA LOADING
# Load experiment results from database or files

# Try to load from run metadata first
experiment_results = run_metadata.get('results', {})
condition_results = experiment_results.get('condition_results', [])

if condition_results:
    print(f"✅ Found {len(condition_results)} model conditions")
    
    # Convert to DataFrame for easy analysis
    model_data = []
    for condition in condition_results:
        model_name = condition.get('condition_identifier', 'Unknown')
        centroid = condition.get('centroid', [0, 0])
        total_analyses = condition.get('total_analyses', 0)
        
        model_data.append({
            'model': model_name,
            'centroid_x': centroid[0] if len(centroid) > 0 else 0,
            'centroid_y': centroid[1] if len(centroid) > 1 else 0,
            'total_analyses': total_analyses,
            'magnitude': np.sqrt(centroid[0]**2 + centroid[1]**2) if len(centroid) >= 2 else 0
        })
    
    # Create DataFrame
    results_df = pd.DataFrame(model_data)
    print("\n📊 Model Results Summary:")
    display(results_df)
    
else:
    print("⚠️ No condition results found")
    print("💡 Creating sample data for template demonstration")
    
    # Create sample data so template can still be used
    results_df = pd.DataFrame({
        'model': ['Sample Model A', 'Sample Model B', 'Sample Model C'],
        'centroid_x': [0.5, -0.3, 0.1],
        'centroid_y': [0.2, 0.7, -0.4],
        'total_analyses': [10, 15, 8],
        'magnitude': [0.54, 0.76, 0.41]
    })
    
    print("\n📊 Sample Data for Template:")
    display(results_df)


In [ ]:
# 🎯 COORDINATE VISUALIZATION
# Universal coordinate plot that adapts to any framework

fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Plot model centroids
colors = sns.color_palette('husl', len(results_df))
for idx, (_, row) in enumerate(results_df.iterrows()):
    ax.scatter(row['centroid_x'], row['centroid_y'], 
              s=200, color=colors[idx], alpha=0.7, 
              label=row['model'], edgecolors='black', linewidth=1)
    
    # Add model labels
    ax.annotate(row['model'], 
               (row['centroid_x'], row['centroid_y']),
               xytext=(5, 5), textcoords='offset points',
               fontsize=10, ha='left')

# Add coordinate system lines
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)

# Labels and title
framework_name = experiment_def.get('framework', {}).get('name', 'Unknown Framework')
ax.set_xlabel('X Coordinate', fontsize=12)
ax.set_ylabel('Y Coordinate', fontsize=12)
ax.set_title(f'Model Comparison: {framework_name}\nCentroid Positions in Coordinate Space', 
            fontsize=14, pad=20)

# Legend and grid
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

print("✅ Coordinate visualization complete")
print("💡 Tip: Modify this cell to customize the visualization")


In [ ]:
# 📁 EXPORT RESULTS
# Save results for publication or further analysis

# Create exports directory
export_dir = Path('exports')
export_dir.mkdir(exist_ok=True)

# Export DataFrame to CSV
results_csv = export_dir / 'model_results.csv'
results_df.to_csv(results_csv, index=False)
print(f"✅ Results exported to: {results_csv}")

# Export visualization
fig_path = export_dir / 'coordinate_visualization.png'
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
for idx, (_, row) in enumerate(results_df.iterrows()):
    ax.scatter(row['centroid_x'], row['centroid_y'], 
              s=200, color=colors[idx], alpha=0.7, 
              label=row['model'], edgecolors='black', linewidth=1)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('X Coordinate', fontsize=12)
ax.set_ylabel('Y Coordinate', fontsize=12)
ax.set_title(f'Model Comparison: {framework_name}', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.close()
print(f"✅ Visualization exported to: {fig_path}")

# Export summary report
report_path = export_dir / 'analysis_summary.txt'
with open(report_path, 'w') as f:
    f.write(f"Experiment Analysis Summary\n")
    f.write(f"=========================\n\n")
    f.write(f"Experiment: {experiment_def.get('name', 'Unknown')}\n")
    f.write(f"Framework: {framework_name}\n")
    f.write(f"Job ID: {run_metadata.get('job_id', 'Unknown')}\n")
    f.write(f"Analysis Date: {datetime.now().isoformat()}\n\n")
    f.write(f"Models Analyzed: {len(results_df)}\n")
    f.write(f"Total Analyses: {results_df['total_analyses'].sum()}\n")
    f.write(f"Mean Magnitude: {results_df['magnitude'].mean():.3f}\n")
    f.write(f"\nModel Details:\n")
    for _, row in results_df.iterrows():
        f.write(f"  {row['model']}: ({row['centroid_x']:.3f}, {row['centroid_y']:.3f})\n")

print(f"✅ Summary report exported to: {report_path}")
print("\n🎉 All exports complete!")
print("💡 Ready for academic publication or further analysis")
